### Instalação e leitura


In [ ]:
!pip install PyMuPDF python-docx sentence-transformers faiss-cpu "pypdf>=3.0.0" "transformers>=4.0.0" "torch>=2.0.0" requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 26.6 MB/s eta 0:00:00


In [ ]:
# -*- coding: utf-8 -*-
"""
Script final para um sistema de Q&A com RAG, com melhorias avançadas de extração:
- Para PDFs: PyMuPDF com chunking baseado em sentenças para maior foco.
- Para DOCX: python-docx com extração de contexto de tabelas para máxima precisão.
- Consulta aos 3 modelos de QA Extrativos exigidos.
"""
import os
import requests
import numpy as np
import faiss
import time
import fitz  # PyMuPDF
import docx # python-docx
import re    # Para divisão de sentenças

# NOVOS IMPORTS PARA A CORREÇÃO DO DOCX
from docx.oxml.table import CT_Tbl
from docx.oxml.text.paragraph import CT_P

from sentence_transformers import SentenceTransformer, CrossEncoder


In [ ]:
CAMINHO_DO_PDF = "doencas_respiratorias_cronicas.pdf"
CAMINHO_DO_DOCX = "DICIONARIO_DE_DADOS.docx"
try:
    from google.colab import userdata
    API_TOKEN = userdata.get('HF_TOKEN')
    if API_TOKEN is None: raise ValueError("Token não encontrado")
    print("✅ Token da Hugging Face carregado com sucesso!")
except (ImportError, KeyError, ValueError):
    API_TOKEN = "" # INSIRA SEU TOKEN DA HUGGING FACE AQUI
    if not API_TOKEN:
        print("🛑 ERRO: Token da Hugging Face não configurado. Insira-o na variável API_TOKEN.")

# --- DICIONÁRIO DE 3 MODELOS DE QA (EXTRATIVOS E LEVES) ---
MODELOS_QA = {
    "mDeBERTa-v3-base-squad2": "https://api-inference.huggingface.co/models/timpal0l/mdeberta-v3-base-squad2",
    "Google-bert": "https://api-inference.huggingface.co/models/google-bert/bert-large-uncased-whole-word-masking-finetuned-squad",
    "RoBERTa-base-squad2": "https://api-inference.huggingface.co/models/deepset/roberta-base-squad2"
}

HEADERS = {"Authorization": f"Bearer {API_TOKEN}"}


✅ Token da Hugging Face carregado com sucesso!


In [ ]:
def query_qa_api(payload, model_url):
    model_name = model_url.split("/")[-1]
    max_retries = 3
    wait_time = 15
    for attempt in range(max_retries):
        try:
            response = requests.post(model_url, headers=HEADERS, json=payload)
            if response.status_code == 503:
                print(f"   ⏳ Modelo '{model_name}' está carregando... aguardando {wait_time}s")
                time.sleep(wait_time)
                continue
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"   🛑 ERRO na chamada da API para '{model_name}': {e}")
            return {"error": str(e)}
    return {"error": f"Modelo '{model_name}' não respondeu."}



In [ ]:
def processar_pdf_com_sentencas(caminho_arquivo):
    """Extrai texto de PDF, dividindo em sentenças e agrupando-as em chunks focados."""
    print("1/4 - Processando PDF com PyMuPDF (chunking por sentenças)...")
    doc = fitz.open(caminho_arquivo)
    texto_completo = "".join([page.get_text("text") for page in doc])
    texto_completo = re.sub(r'\s*\n\s*', ' ', texto_completo)
    sentencas = re.split(r'(?<=[.!?])\s+', texto_completo)

    chunks_com_metadata = []
    sentencas_por_chunk = 4
    overlap = 1

    for i in range(0, len(sentencas), sentencas_por_chunk - overlap):
        grupo_sentencas = sentencas[i : i + sentencas_por_chunk]
        chunk_texto = " ".join(grupo_sentencas).strip()
        if len(chunk_texto.split()) > 10:
            chunks_com_metadata.append({"text": chunk_texto, "metadata": {"page": "N/A"}})

    print(f"✅ PDF processado. {len(chunks_com_metadata)} chunks de sentenças criados.")
    return chunks_com_metadata

def processar_docx_com_contexto_tabela(caminho_arquivo):
    """Extrai dados de tabelas de um DOCX, adicionando o nome da tabela como contexto."""
    print("1/4 - Processando DOCX com python-docx (com contexto de tabela)...")
    document = docx.Document(caminho_arquivo)
    chunks_com_metadata = []

    # Itera sobre os elementos do corpo do documento (parágrafos e tabelas)
    for i, block in enumerate(document.element.body):


        if not isinstance(block, CT_Tbl):
            continue


        table = docx.table.Table(block, document)
        contexto_tabela = "Contexto não identificado"

        # Procura por um parágrafo imediatamente antes da tabela para usar como título
        if i > 0 and isinstance(document.element.body[i-1], CT_P):
            paragrafo_anterior = docx.text.paragraph.Paragraph(document.element.body[i-1], document)
            if paragrafo_anterior.text.strip():
                texto_paragrafo = " ".join(paragrafo_anterior.text.strip().split())
                match = re.search(r'LFCES\d+,\s*(\w+)', texto_paragrafo)
                if match:
                    contexto_tabela = match.group(1)
                else:
                    contexto_tabela = texto_paragrafo

        for j, row in enumerate(table.rows):
            if j == 0: continue

            try:
                nome_campo = row.cells[0].text.strip()
                descricao = row.cells[8].text.strip()
                dominios = row.cells[9].text.strip()

                if nome_campo and descricao:
                    sentenca = f"Na tabela '{contexto_tabela}', o campo '{nome_campo}' é descrito como: '{descricao}'."
                    if dominios:
                        sentenca += f" Seus domínios ou valores possíveis são: '{dominios}'."

                    chunks_com_metadata.append({"text": sentenca, "metadata": {"page": "N/A"}})
            except IndexError:
                continue

    print(f"✅ DOCX processado. {len(chunks_com_metadata)} chunks (sentenças com contexto) criados.")
    return chunks_com_metadata

def indexar_documento(caminho_documento):
    """Função unificada para processar e indexar qualquer tipo de documento suportado."""
    print(f"\n--- INICIANDO INDEXAÇÃO PARA: {os.path.basename(caminho_documento)} ---")
    if not os.path.exists(caminho_documento):
        print(f"🛑 ERRO: Arquivo não encontrado.")
        return None, None, None

    if caminho_documento.lower().endswith('.pdf'):
        chunks_com_metadata = processar_pdf_com_sentencas(caminho_documento)
    elif caminho_documento.lower().endswith('.docx'):
        chunks_com_metadata = processar_docx_com_contexto_tabela(caminho_documento)
    else:
        print(f"🛑 ERRO: Formato de arquivo não suportado.")
        return None, None, None

    if not chunks_com_metadata:
        print("🛑 Nenhuma informação válida foi extraída do documento.")
        return None, None, None

    textos_para_embeddar = [chunk['text'] for chunk in chunks_com_metadata]

    print("2/4 - Gerando embeddings...")
    embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = embedding_model.encode(textos_para_embeddar, show_progress_bar=True)

    print("3/4 - Criando índice FAISS...")
    faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
    faiss.normalize_L2(embeddings)
    faiss_index.add(np.array(embeddings, dtype=np.float32))

    print(f"✅ Índice FAISS criado com {faiss_index.ntotal} vetores.")
    print("--- FASE DE INDEXAÇÃO CONCLUÍDA ---")
    return faiss_index, chunks_com_metadata, embedding_model


In [ ]:
def executar_qa(faiss_index, chunks, embedding_model, cross_encoder_model, lista_de_perguntas, nome_documento):
    if not all([faiss_index, chunks, embedding_model, cross_encoder_model]):
        print(f"\nProcesso interrompido para '{nome_documento}' por erro na indexação.")
        return

    print(f"\n\n--- INICIANDO CONSULTA PARA: {nome_documento} ---")
    for pergunta in lista_de_perguntas:
        start_time = time.time()
        print(f"\n\n=================================================")
        print(f'❓ Pergunta: "{pergunta}"')
        print(f"=================================================")

        print("1. Buscando e re-classificando chunks...")
        query_embedding = embedding_model.encode([pergunta])
        faiss.normalize_L2(query_embedding)
        k_retrieval = 15
        _, indices = faiss_index.search(np.array(query_embedding, dtype=np.float32), k_retrieval)
        retrieved_chunks_initial = [chunks[i] for i in indices[0]]
        cross_encoder_input = [[pergunta, chunk['text']] for chunk in retrieved_chunks_initial]
        scores = cross_encoder_model.predict(cross_encoder_input, show_progress_bar=False)
        chunk_score_pairs = sorted(zip(retrieved_chunks_initial, scores), key=lambda x: x[1], reverse=True)

        k_final = 1
        reranked_chunks = [pair[0] for pair in chunk_score_pairs[:k_final] if pair[1] > -5.0]

        if not reranked_chunks:
            print("❌ Não foram encontrados chunks relevantes.")
            continue

        contexto_final = "\n---\n".join([chunk['text'] for chunk in reranked_chunks])
        fontes = sorted(list(set([chunk['metadata']['page'] for chunk in reranked_chunks])))
        print(f"✅ Contexto final selecionado (fontes: {fontes}).")

        payload = {"inputs": {"question": pergunta, "context": contexto_final}}
        print("\n2. Consultando modelos de IA...")
        for nome_modelo, url_modelo in MODELOS_QA.items():
            print(f"\n--- Consultando o modelo: {nome_modelo} ---")
            resultado = query_qa_api(payload, url_modelo)
            if 'answer' in resultado and resultado['answer'] is not None:
                print(f"✅ Resposta: {resultado['answer']}")
                print(f"   Confiança (Score): {resultado.get('score', 0):.4f}")
            else:
                print(f"❌ Erro ou resposta não encontrada: {resultado}")
            time.sleep(1)

        end_time = time.time()
        print(f"\n(Tempo total para esta pergunta: {end_time - start_time:.2f} segundos)")



In [ ]:
if __name__ == "__main__":
    print("\nCarregando modelo Cross-Encoder para re-ranking...")
    cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
    print("✅ Cross-Encoder carregado.")

    # --- PROCESSO PARA O ARQUIVO PDF ---
    faiss_index_pdf, chunks_pdf, embedding_model_pdf = indexar_documento(CAMINHO_DO_PDF)
    if faiss_index_pdf:
        lista_de_perguntas_pdf = [
            "Estou sentindo falta de ar, tossindo e com dor no peito. Qual doença possui esses sintomas?",
            "Quais são os sintomas da rinite alérgica?",
            "Como posso determinar se tenho alguma doença respiratória crônica?"
        ]
        executar_qa(faiss_index_pdf, chunks_pdf, embedding_model_pdf, cross_encoder_model, lista_de_perguntas_pdf, os.path.basename(CAMINHO_DO_PDF))

    # --- PROCESSO PARA O ARQUIVO DOCX ---
    faiss_index_docx, chunks_docx, embedding_model_docx = indexar_documento(CAMINHO_DO_DOCX)
    if faiss_index_docx:
        lista_de_perguntas_docx = [
            "Qual a descrição e os domínios do campo STATUSMOV na tabela de estabelecimentos de saúde?",
            "Qual a descrição do campo NSLAQCDURA na tabela de quimioterapia e radioterapia?",
            "Qual a descrição da tabela LFCES008?"
        ]
        executar_qa(faiss_index_docx, chunks_docx, embedding_model_docx, cross_encoder_model, lista_de_perguntas_docx, os.path.basename(CAMINHO_DO_DOCX))


Carregando modelo Cross-Encoder para re-ranking...
✅ Cross-Encoder carregado.

--- INICIANDO INDEXAÇÃO PARA: doencas_respiratorias_cronicas.pdf ---
🛑 ERRO: Arquivo não encontrado.

--- INICIANDO INDEXAÇÃO PARA: DICIONARIO_DE_DADOS.docx ---
🛑 ERRO: Arquivo não encontrado.
